<a href="https://colab.research.google.com/github/Nirmalbhatii/CMCA546_NLP/blob/main/S25MCAG0039_LAB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import nltk
from nltk.corpus import brown

nltk.download('brown')
nltk.download('punkt')

news_words = brown.words(categories='news')
news_sents = brown.sents(categories='news')

words = [w.lower() for w in news_words if w.isalpha()]

print("Total sentences:", len(news_sents))
print("Total words:", len(words))
print("Vocabulary size:", len(set(words)))


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total sentences: 4623
Total words: 83562
Vocabulary size: 11151


In [12]:
def ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

unigrams = ngrams(words, 1)
bigrams = ngrams(words, 2)
trigrams = ngrams(words, 3)

bigram_freq = Counter(bigrams)
trigram_freq = Counter(trigrams)

print("Q2 Results:")
print("Top 20 Bigrams:")
for bg, count in bigram_freq.most_common(20):
    print(bg, ":", count)

print("\nTop 20 Trigrams:")
for tg, count in trigram_freq.most_common(20):
    print(tg, ":", count)
print("-" * 50)


Q2 Results:
Top 20 Bigrams:
('of', 'the') : 850
('in', 'the') : 610
('to', 'the') : 279
('on', 'the') : 258
('for', 'the') : 223
('at', 'the') : 204
('will', 'be') : 157
('that', 'the') : 149
('and', 'the') : 142
('with', 'the') : 142
('in', 'a') : 120
('of', 'a') : 120
('by', 'the') : 115
('to', 'be') : 108
('from', 'the') : 104
('for', 'a') : 101
('as', 'a') : 91
('the', 'first') : 88
('has', 'been') : 87
('he', 'said') : 80

Top 20 Trigrams:
('one', 'of', 'the') : 44
('the', 'united', 'states') : 34
('members', 'of', 'the') : 28
('president', 'of', 'the') : 22
('a', 'number', 'of') : 19
('the', 'white', 'house') : 19
('as', 'a', 'result') : 18
('some', 'of', 'the') : 18
('chairman', 'of', 'the') : 16
('of', 'the', 'new') : 16
('per', 'cent', 'of') : 15
('a', 'member', 'of') : 15
('in', 'the', 'past') : 14
('the', 'new', 'york') : 14
('will', 'be', 'the') : 13
('the', 'university', 'of') : 13
('as', 'well', 'as') : 13
('a', 'result', 'of') : 12
('in', 'the', 'state') : 12
('for', 'th

In [13]:
def bigram_prob(w1, w2):
    bigram_count = bigram_freq[(w1, w2)]
    unigram_count = words.count(w1)
    return bigram_count / unigram_count if unigram_count > 0 else 0

def trigram_prob(w1, w2, w3):
    trigram_count = trigram_freq[(w1, w2, w3)]
    bigram_count = bigram_freq[(w1, w2)]
    return trigram_count / bigram_count if bigram_count > 0 else 0

print("Q3 Results:")
print("P('the' | 'in') =", bigram_prob("in", "the"))
print("P('company' | 'of', 'the') =", trigram_prob("of", "the", "company"))
print("-" * 50)

Q3 Results:
P('the' | 'in') = 0.30198019801980197
P('company' | 'of', 'the') = 0.001176470588235294
--------------------------------------------------


In [15]:
import re

def tokenize_like_corpus(s: str):
    return re.findall(r"[a-z]+", s.lower())


V = len(set(words))


def bigram_prob_smoothed(w1, w2):
    return (bigram_freq[(w1, w2)] + 1) / (words.count(w1) + V)

def trigram_prob_smoothed(w1, w2, w3):
    return (trigram_freq[(w1, w2, w3)] + 1) / (bigram_freq[(w1, w2)] + V)

def sentence_prob_bigram(sentence: str):
    toks = tokenize_like_corpus(sentence)
    if len(toks) < 2:
        return 0.0
    p = 1.0
    for i in range(len(toks) - 1):
        p *= bigram_prob_smoothed(toks[i], toks[i+1])
    return p

def sentence_prob_trigram(sentence: str):
    toks = tokenize_like_corpus(sentence)
    if len(toks) < 3:

        return sentence_prob_bigram(sentence)
    p = 1.0
    for i in range(len(toks) - 2):
        p *= trigram_prob_smoothed(toks[i], toks[i+1], toks[i+2])
    return p

sentence = "the president of the company"
print("Q4 Results (smoothed):")
print("Bigram Model Probability:", sentence_prob_bigram(sentence))
print("Trigram Model Probability:", sentence_prob_trigram(sentence))


Q4 Results (smoothed):
Bigram Model Probability: 4.146986564495009e-10
Trigram Model Probability: 9.181723409951398e-11
